## KOSIS 데이터 수집

In [1]:
import time
from tqdm import tqdm
import requests

import pandas as pd
# import pandas.io.json._normalize as json_normalize

### 함수

In [85]:
def get_kosis_api_data(url, print_elapsed=True):
    """ '통계표선택'을 통해 생성한 API 요청 url을 받고, 데이터를 받아와 정리 후 통계표명과 데이터프레임으로 반환한다.
    """
    ## 요청-반환 ##
    ti = time.time()
    res = requests.get(url)
    if print_elapsed:
        print("API GET Request Elapsed:", round(time.time() - ti, 2))

    ## 데이터프레임 생성 ##
    data = res.json()
    df = pd.json_normalize(data)
    
    ## 컬럼 1차 필터링 ##
    cols = [col for col in df.columns if "ENG" not in col]  # 영문컬럼 삭제
    df = df.loc[:, cols]
    
    ## 컬럼명 수정 ##
    renames = {}

    # 통계표명
    renames["TBL_NM"] = "통계표"
    # 시점 관련
    renames["PRD_DE"] = "시점"
    renames["PRD_SE"] = "주기_코드"
    # 데이터 및 분류 관련
    renames["DT"] = "값"
    renames["ITM_NM"] = "항목명"
    renames["UNIT_NM"] = "단위"

    # 함수
    get_col_single_value = lambda df, col_name: "".join(set(df[col_name].to_list()))  # 해당 컬럼의 값들을 중복 제거하고 하나의 문자열로 반환

    # 카테고리들 컬럼명 변경 (함수 이용)
    cat_obj_cols = [col for col in df.columns if "C" in col and "OBJ" in col]
    for cat_obj in cat_obj_cols:
        origin_name = cat_obj.replace("_OBJ", "")  #  OBJ 뺀 컬럼 이름
        new_name = get_col_single_value(df, cat_obj)
        renames[origin_name] = new_name
        
        origin_code_name = origin_name.replace("_NM", "")
        new_code_name = f"{new_name}_코드"
        renames[origin_code_name] = new_code_name

    # 컬럼명 수정
    df.rename(renames, axis=1, inplace=True)
    
    # print(df)
    
    ## 컬럼명 수정 후, 수정한 컬럼들만 남기기 ##
    # renames_targets = renames.values()
    # for rm in df.columns:
    #     if rm in renames_targets:
    #         renames_targets.remove(rm)
    
    last_cols = ["항목명", "값", "단위"] 
    cols = [col for col in renames.values() if col not in last_cols] + last_cols
    # for rm in renames_targets:
    #     if rm in cols:
    #         cols.remove(rm)
    df = df.loc[:, cols]
    
    ## 반환 ##
    stat_name = get_col_single_value(df, "통계표")
    
    return stat_name, df


def collect_kosis_api_data_with_date_range(url_base, date_cycle="연간", start_year=2019, end_year=2021, pass_error=True):
    """kosis api를 통해 기간의 데이터를 수집함
    
    Args:
        url_base(str): 통계표선택에서 날짜 지정 수집으로 가져온 url에서 날짜 부분을 {date}로 변경한 것 (format을 이용해서 date를 지정함)
        ~
    
    Returns:
        저장할 때 쓸 수 있는 파일명(csv)과 데이터프레임 반환
    """
    # date_cycle 유형에 따라 날짜를 만들기 (date_cycle, start_yr, end_yr 이용)
    if date_cycle == "연간":
        elements = [""]
    elif date_cycle == "반기":
        elements = ["01", "02"]
    elif date_cycle == "월간":
        elements = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
    elif date_cycle == "분기":
        elements = ["01", "02", "03", "04"]
    else:
        elements = [""]
    
    dates = []
    for yr in range(start_year, end_year + 1):
        for ele in elements:
            dates.append(f"{yr}{ele}")
    
    # 실행
    dfs = []
    for d in tqdm(dates):
        # url에서 startPreDe와 endPreDe를 d로 수정 (={date}로 바꿔놓은것 format으로 넣기)
        if pass_error:
            try:
                name, df = get_kosis_api_data(url_base.format(date=d), print_elapsed=False)  # 날짜만 다르게 해서 수집하는 것
                dfs.append(df)
            except Exception as e:
                print("Error: ", e)
        else:
            name, df = get_kosis_api_data(url_base.format(date=d), print_elapsed=False)  # 날짜만 다르게 해서 수집하는 것
            dfs.append(df)
            
    # 데이터프레임 병합
    filename = f"{name}_{date_cycle}_{dates[0]}-{dates[-1]}.csv"  # 저장하려면 이 파일명으로 저장하기
    result = pd.concat(dfs)
    result["주기"] = date_cycle
    result.reset_index(drop=True, inplace=True)
    
    return filename, result


def to_float(x, to_none=["-"]):
    if x in to_none:
        return None
    else:
        return float(x)

### 수집

In [86]:
# 리스트 안에 [url_base(날짜부분 date로 변경), date_cycle(연간/월간/반기), start_yr, end_yr] 순으로 넣기
to_collects = [
    # [url_base,
    # date_cycle, start_yr, end_yr]
    
    # # 행정구역(시군구)별 성별 인구수_연간
    # ["https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T20+T21+T22+&objL1=11110+11140+11170+11200+11215+11230+11260+11290+11305+11320+11350+11380+11410+11440+11470+11500+11530+11545+11560+11590+11620+11650+11680+11710+11740+26110+26140+26170+26200+26230+26260+26290+26320+26350+26380+26410+26440+26470+26500+26530+26710+27110+27140+27170+27200+27230+27260+27290+27710+28110+28114+28118+28140+28170+28177+28185+28200+28230+28237+28245+28260+28265+28710+28720+29110+29140+29155+29170+29200+30110+30140+30170+30200+30230+31110+31140+31170+31200+31710+36110+41110+41111+41113+41115+41117+41130+41131+41133+41135+41150+41170+41171+41173+41190+41195+41197+41199+41210+41220+41222+41224+41230+41250+41270+41271+41273+41280+41281+41283+41285+41287+41290+41310+41350+41360+41362+41370+41390+41410+41430+41450+41460+41461+41463+41465+41480+41500+41550+41570+41590+41592+41594+41610+41630+41650+41670+41710+41720+41730+41740+41750+41770+41790+41800+41810+41820+41830+41840+41850+41860+41870+41880+41890+42110+42130+42150+42170+42190+42210+42230+42715+42720+42730+42745+42750+42760+42770+42780+42790+42800+42810+42820+42830+42840+42850+43110+43112+43114+43111+43113+43115+43116+43130+43150+43710+43720+43730+43740+43745+43750+43760+43770+43780+43795+43800+43950+44270+44130+44131+44133+44150+44170+44180+44190+44200+44210+44220+44230+44250+44710+44730+44740+44750+44760+44770+44780+44790+44800+44810+44820+44825+44830+44840+44860+45110+45111+45113+45118+45130+45140+45145+45180+45190+45210+45710+45720+45730+45740+45750+45760+45770+45780+45790+45800+45810+45820+45830+46110+46130+46150+46170+46190+46210+46230+46710+46720+46730+46740+46750+46760+46770+46780+46790+46800+46810+46820+46830+46840+46850+46860+46870+46880+46890+46900+46910+47110+47111+47113+47130+47150+47170+47190+47210+47230+47250+47270+47280+47290+47710+47720+47730+47740+47750+47760+47770+47780+47795+47800+47810+47820+47830+47840+47850+47860+47870+47880+47890+47900+47910+47920+47930+47940+48120+48121+48123+48125+48127+48129+48110+48130+48151+48153+48160+48170+48190+48210+48220+48230+48240+48245+48250+48252+48270+48290+48310+48330+48332+48710+48720+48730+48740+48750+48760+48775+48780+48795+48800+48810+48820+48830+48840+48850+48860+48870+48880+48890+50110+50130+50910+50920+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=101&tblId=DT_1B040A3",
    #  "연간", 2019, 2021],
    # # 시군구별 종사자수
    # ["https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=16142T209+16142118+16142119+16142117+16142121+16142120+&objL1=15142N.00+15142N.11+15142N.11010+15142N.11020+15142N.11030+15142N.11040+15142N.11050+15142N.11060+15142N.11070+15142N.11080+15142N.11090+15142N.11100+15142N.11110+15142N.11120+15142N.11130+15142N.11140+15142N.11150+15142N.11160+15142N.11170+15142N.11180+15142N.11190+15142N.11200+15142N.11210+15142N.11220+15142N.11230+15142N.11240+15142N.11250+15142N.21+15142N.21010+15142N.21020+15142N.21030+15142N.21040+15142N.21050+15142N.21060+15142N.21070+15142N.21080+15142N.21090+15142N.21100+15142N.21110+15142N.21120+15142N.21130+15142N.21140+15142N.21150+15142N.21310+15142N.22+15142N.22010+15142N.22020+15142N.22030+15142N.22040+15142N.22050+15142N.22060+15142N.22070+15142N.22310+15142N.23+15142N.23010+15142N.23020+15142N.23040+15142N.23050+15142N.23060+15142N.23070+15142N.23080+15142n.23090+15142N.23310+15142N.23320+15142N.24+15142N.24010+15142N.24020+15142N.24030+15142N.24040+15142N.24050+15142N.25+15142N.25010+15142N.25020+15142N.25030+15142N.25040+15142N.25050+15142N.26+15142N.26010+15142N.26020+15142N.26030+15142N.26040+15142N.26310+15142N.43+15142N.43010+15142N.31+15142N.31011+15142N.31012+15142N.31013+15142N.31014+15142N.31021+15142N.31022+15142N.31023+15142N.31030+15142N.31041+15142N.31042+15142N.31051+15142N.31060+15142N.31070+15142N.31080+15142N.31091+15142N.31092+15142N.31101+15142N.31103+15142N.31104+15142N.31110+15142N.31120+15142N.31130+15142N.31140+15142N.31150+15142N.31160+15142N.31170+15142N.31180+15142N.31191+15142N.31192+15142N.31193+15142N.31200+15142N.31210+15142N.31220+15142N.31230+15142N.31240+15142N.31250+15142N.31260+15142N.31270+15142N.31320+15142N.31350+15142N.31370+15142N.31380+15142N.32+15142N.32010+15142N.32020+15142N.32030+15142N.32040+15142N.32050+15142N.32060+15142N.32070+15142N.32310+15142N.32320+15142N.32330+15142N.32340+15142N.32350+15142N.32360+15142N.32370+15142N.32380+15142N.32390+15142N.32400+15142N.32410+15142N.33+15142N.33020+15142N.33030+15142N.33011+15142N.33402+15142N.33012+15142N.33310+15142N.33320+15142N.33330+15142N.33340+15142N.33350+15142N.33360+15142N.33370+15142N.33380+15142N.33390+15142N.34+15142N.34011+15142N.34012+15142N.34020+15142N.34030+15142N.34040+15142N.34050+15142N.34060+15142N.34070+15142N.34390+15142N.34310+15142N.34330+15142N.34340+15142N.34350+15142N.34360+15142N.34370+15142N.34380+15142N.35+15142N.35011+15142N.35012+15142N.35020+15142N.35030+15142N.35040+15142N.35050+15142N.35060+15142N.35310+15142N.35320+15142N.35330+15142N.35340+15142N.35350+15142N.35360+15142N.35370+15142N.35380+15142N.36+15142N.36010+15142N.36020+15142N.36030+15142N.36040+15142N.36060+15142N.36310+15142N.36320+15142N.36330+15142N.36350+15142N.36360+15142N.36370+15142N.36380+15142N.36390+15142N.36400+15142N.36410+15142N.36420+15142N.36430+15142N.36440+15142N.36450+15142N.36460+15142N.36470+15142N.36480+15142N.37+15142N.37011+15142N.37012+15142N.37020+15142N.37030+15142N.37040+15142N.37050+15142N.37060+15142N.37070+15142N.37080+15142N.37090+15142N.37100+15142N.37310+15142N.37320+15142N.37330+15142N.37340+15142N.37350+15142N.37360+15142N.37370+15142N.37380+15142N.37390+15142N.37400+15142N.37410+15142N.37420+15142N.37421+15142N.38+15142N.38010+15142N.38020+15142N.38030+15142N.38040+15142N.38050+15142N.38060+15142N.38070+15142N.38080+15142N.38090+15142N.38100+15142N.38310+15142N.38320+15142N.38330+15142N.38340+15142N.38350+15142N.38360+15142N.38370+15142N.38380+15142N.38390+15142N.38400+15142N.38500+15142N.38600+15142N.39+15142N.39010+15142N.39020+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=142&tblId=DT_BR225",
    #  "연간", 2019, 2019],
    # # 용도별 차종별 시군구별 자동차주행거리_연간
    # ["https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T001+T002+&objL1=11010+11020+11030+11040+11050+11060+11070+11080+11090+11100+11110+11120+11130+11140+11150+11160+11170+11180+11190+11200+11210+11220+11230+11240+11250+21010+21020+21030+21040+21050+21060+21070+21080+21090+21100+21110+21120+21130+21140+21150+21310+22010+22020+22030+22040+22050+22060+22070+22310+23010+23020+23030+23040+23050+23060+23070+23080+23310+23320+24010+24020+24030+24040+24050+25010+25020+25030+25040+25050+26010+26020+26030+26040+26310+29010+31010+31020+31030+31040+31050+31060+31070+31080+31090+31100+31110+31120+31130+31140+31150+31160+31170+31180+31190+31200+31210+31220+31230+31240+31250+31260+31270+31280+31350+31370+31380+32010+32020+32030+32040+32050+32060+32070+32310+32320+32330+32340+32350+32360+32370+32380+32390+32400+32410+33010+33020+33030+33310+33320+33330+33340+33390+33350+33360+33370+33380+34010+34020+34030+34040+34050+34060+34070+34080+34310+34320+34330+34340+34350+34360+34370+34380+35010+35020+35030+35040+35050+35060+35310+35320+35330+35340+35350+35360+35370+35380+36010+36020+36030+36040+36060+36310+36320+36330+36350+36360+36370+36380+36390+36400+36410+36420+36430+36440+36450+36460+36470+36480+37010+37020+37030+37040+37050+37060+37070+37080+37090+37100+37310+37320+37330+37340+37350+37360+37370+37380+37390+37400+37410+37420+37430+38110+38030+38050+38060+38070+38080+38090+38100+38310+38320+38330+38340+38350+38360+38370+38380+38390+38400+39010+39020+&objL2=0001+0002+0003+&objL3=0001+0002+0003+0004+0005+&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=426&tblId=DT_426001_N004",
    #  "연간", 2019, 2020],
    # # 시군구/성/월별 출생
    # ["https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T1+T2+T3+&objL1=00+11+11010+11020+11030+11040+11050+11060+11070+11080+11090+11100+11110+11120+11130+11140+11150+11160+11170+11180+11190+11200+11210+11220+11230+11240+11250+21+21010+21020+21030+21040+21050+21060+21070+21080+21090+21100+21110+21120+21130+21140+21150+21310+22+22010+22020+22030+22040+22050+22060+22070+22310+23+23010+23020+23030+23040+23050+23060+23070+23080+23310+23320+23090+24+24010+24020+24030+24040+24050+25+25010+25020+25030+25040+25050+26+26010+26020+26030+26040+26310+29+29010+31+31010+31011+31012+31013+31014+31020+31021+31022+31023+31030+31040+31041+31042+31050+31051+31052+31053+31060+31070+31080+31090+31091+31092+31100+31101+31102+31103+31104+31110+31120+31130+31140+31150+31160+31170+31180+31190+31191+31192+31193+31200+31210+31220+31230+31240+31250+31260+31270+31280+31310+31320+31330+31340+31350+31360+31370+31380+31390+31400+32+32010+32020+32030+32040+32050+32060+32070+32310+32320+32330+32340+32350+32360+32370+32380+32390+32400+32410+33+33010+33011+33012+33020+33030+33040+33041+33042+33043+33044+33310+33320+33330+33340+33350+33360+33370+33380+33390+34+34010+34011+34012+34020+34030+34040+34050+34060+34070+34080+34310+34320+34330+34340+34350+34360+34370+34380+34390+35+35010+35011+35012+35020+35030+35040+35050+35060+35310+35320+35330+35340+35350+35360+35370+35380+36+36010+36020+36030+36040+36050+36060+36310+36320+36330+36340+36350+36360+36370+36380+36390+36400+36410+36420+36430+36440+36450+36460+36470+36480+37+37010+37011+37012+37020+37030+37040+37050+37060+37070+37080+37090+37100+37310+37320+37330+37340+37350+37360+37370+37380+37390+37400+37410+37420+37430+38+38010+38020+38021+38022+38030+38040+38050+38060+38070+38080+38090+38100+38110+38111+38112+38113+38114+38115+38310+38320+38330+38340+38350+38360+38370+38380+38390+38400+39+39010+39020+39310+39320+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=101&tblId=DT_1B81A01",
    #  "연간", 2019, 2020],
    # # 연령대별 소득
    # ["https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T001+&objL1=000+10+20+30+40+50+60+70+80+90+100+110+120+&objL2=00+10+20+30+40+50+60+70+80+90+100+110+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=101&tblId=DT_1EP_2006",
    #  "연간", 2019, 2020],
    # # 성별 연령대별 소득
    # ["https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T001+T002+&objL1=0+1+2+&objL2=00+10+20+30+40+50+60+70+80+90+100+110+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=101&tblId=DT_1EP_2010",
    #  "연간", 2019, 2020],
    # # 향후 선호하는 금융상품의 형태
    # ["https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T1+&objL1=A001+B003+1004+2005+3006+4007+5008+6009+I052+1053+2054+3055+4056+5057+6058+7059+H045+1046+2047+3048+4049+5050+6051+6052+T085+1086+2087+PP186+1187+2188+3189+J060+1061+2063+3065+4067+5069+6071+7073+8075+9077+V096+2097+3098+4099+5100+E028+1029+2030+C010+1011+2012+3013+4014+5015+D016+1017+2018+3019+4020+5021+6022+7023+8024+9025+10028+11027+G040+2041+3042+4043+5044+BBB100+1111+2222+3333+4444+5555+CCC100+1112+2223+3334+4445+5556+6667+7778+8889+9990+9999+DDD100+1113+2224+3335+4446+S078+1079+2080+3081+4082+5083+6084+U088+1089+2090+3091+4092+5093+6094+7095+&objL2=1220+101221+102222+103223+104224+105225+106226+108228+107227+109228+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=367&tblId=DT_36701_18_A068",
    #  "연간", 2018, 2021],  # 엄연히 말하면 연간은 아님 3년간임
    # # 전국 산업 연령 교육정도 종사지위별 취업자 - 직업별
    # ["https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T00+T001+T20+T201+T21+T211+T22+T221+T23+T231+T24+T241+T30+T301+T31+T311+T32+T321+T40+T401+T41+T42+T43+T431+T50+T501+&objL1=0+1+11+12+13+14+15+2+21+22+23+24+25+26+27+28+3+31+32+33+39+4+41+42+43+44+5+51+52+53+6+61+62+63+7+71+72+73+74+75+76+77+78+79+8+81+82+83+84+85+86+87+88+89+9+91+92+93+94+95+99+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=H&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=101&tblId=DT_1ES3B28S",
    #  "반기", 2019, 2021],
    # # 전국 산업 연령 교육정도 종사지위별 취업자 - 산업별
    # ["https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T00+T001+T20+T201+T21+T211+T22+T221+T23+T231+T24+T241+T30+T301+T31+T311+T32+T321+T40+T401+T41+T42+T43+T431+T50+T501+&objL1=0+A+A01+A02+A03+B+B05+B06+B07+B08+C+C10+C11+C12+C13+C14+C15+C16+C17+C18+C19+C20+C21+C22+C23+C24+C25+C26+C27+C28+C29+C30+C31+C32+C33+C34+D+D35+E+E36+E37+E38+E39+F+F41+F42+G+G45+G46+G47+H+H49+H50+H51+H52+I+I55+I56+J+J58+J59+J60+J61+J62+J63+K+K64+K65+K66+L+L68+M+M70+M71+M72+M73+N+N74+N75+N76+O+O84+P+P85+Q+Q86+Q87+R+R90+R91+S+S94+S95+S96+T+T97+T98+U+U99+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=H&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=101&tblId=DT_1ES3B27S",
    #  "반기", 2019, 2021],
    # # 시도 성 연령 교육정도별 취업자
    # ["https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T00+T10+T12+T13+T14+T15+T21+T22+T23+&objL1=0000+1100+2100+2200+2300+2400+2500+2600+2900+3100+3200+3300+3400+3500+3600+3700+3800+3900+&objL2=S0+S1+S2+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=H&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=101&tblId=DT_1ES3B04S",
    #  "반기", 2019, 2021],
    # # 종합소득세 주요항목 신고 현황Ⅱ 시·군·구
    # ["https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T001+&objL1=A01+A02+A0201+A0202+A0203+A0204+A0205+A0206+A0207+A0208+A0209+A0210+A0211+A0212+A0213+A0214+A0215+A0216+A0217+A0218+A0219+A0220+A0221+A0222+A0223+A0224+A0225+A03+A0301+A0302+A0303+A0304+A0305+A0306+A0307+A0308+A0309+A0310+A04+A0401+A0402+A0403+A0404+A0405+A0406+A0407+A0408+A0409+A0410+A0411+A0412+A0413+A0414+A0415+A0416+A0417+A0418+A0419+A0420+A0421+A0422+A0423+A0424+A0425+A0426+A0427+A0428+A0429+A0430+A0431+A05+A0501+A0502+A0503+A0504+A0505+A0506+A0507+A0508+A0509+A0510+A0511+A0512+A0513+A0514+A0515+A0516+A0517+A0518+A06+A0601+A0602+A0603+A0604+A0605+A07+A0701+A0702+A0703+A0704+A0705+A0706+A0707+A0708+A0709+A0710+A0711+A08+A0801+A0802+A0803+A0804+A0805+A0806+A0807+A0808+A0809+A0810+A0811+A0812+A0813+A0814+A0815+A09+A10+A1001+A1002+A1003+A1004+A1005+A11+A1101+A1102+A1103+A1104+A1105+A1106+A1107+A1108+A1109+A1110+A1111+A1112+A1113+A1114+A12+A1201+A1202+A1203+A1204+A1205+A1206+A1207+A1208+A1209+A1210+A1211+A1212+A1213+A1214+A1215+A1216+A1217+A1218+A1219+A1220+A1221+A1222+A13+A1301+A1302+A1303+A1304+A1305+A1306+A1307+A1308+A14+A1401+A1402+A1403+A1404+A1405+A1406+A1407+A1408+A1409+A1410+A1411+A1412+A1413+A1414+A1415+A1416+A1417+A1418+A1419+A1420+A1421+A1422+A1423+A15+A1501+A1502+A1503+A1504+A1505+A1506+A1507+A1508+A1509+A1510+A1511+A1512+A1513+A1514+A1515+A1516+A16+A1601+A1602+A1603+A1604+A1605+A17+A1701+A1702+A1703+A1704+A1705+A1706+A1707+A1708+A1709+A1710+A1711+A1712+A1713+A1714+A1715+A1716+A1717+A1718+A18+A1802+A1801+&objL2=15133SGH02+15133SGH0E+15133SGH0F+15133SGH0H+15133SGH0I+15133SGH0J+15133SGH0M+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=133&tblId=DT_133N_A3212",
    #  "연간", 2019, 2020],
    # 시군구/인구동태건수 및 동태율(출생,사망,혼인,이혼)
    # ["https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T10+T11+T20+T22+T30+T31+T40+T41+T50+T51+&objL1=00+11+11010+11020+11030+11040+11050+11060+11070+11080+11090+11100+11110+11120+11130+11140+11150+11160+11170+11180+11190+11200+11210+11220+11230+11240+11250+21+21010+21020+21030+21040+21050+21060+21070+21080+21090+21100+21110+21120+21130+21140+21150+21310+22+22010+22020+22030+22040+22050+22060+22070+22310+23+23010+23020+23030+23040+23050+23060+23070+23080+23310+23320+23090+24+24010+24020+24030+24040+24050+25+25010+25020+25030+25040+25050+26+26010+26020+26030+26040+26310+29+29010+31+31010+31011+31012+31013+31014+31020+31021+31022+31023+31030+31040+31041+31042+31050+31051+31052+31053+31060+31070+31080+31090+31091+31092+31100+31101+31102+31103+31104+31110+31120+31130+31140+31150+31160+31170+31180+31190+31191+31192+31193+31200+31210+31220+31230+31240+31250+31260+31270+31280+31310+31320+31330+31340+31350+31360+31370+31380+32+32010+32020+32030+32040+32050+32060+32070+32310+32320+32330+32340+32350+32360+32370+32380+32390+32400+32410+33+33010+33011+33012+33020+33030+33040+33041+33042+33043+33044+33310+33320+33330+33340+33350+33360+33370+33380+33390+34+34010+34011+34012+34020+34030+34040+34050+34060+34070+34080+34310+34320+34330+34340+34350+34360+34370+34380+34390+35+35010+35011+35012+35020+35030+35040+35050+35060+35310+35320+35330+35340+35350+35360+35370+35380+36+36010+36020+36030+36040+36060+36310+36320+36330+36350+36360+36370+36380+36390+36400+36410+36420+36430+36440+36450+36460+36470+36480+37+37010+37011+37012+37020+37030+37040+37050+37060+37070+37080+37090+37100+37310+37320+37330+37340+37350+37360+37370+37380+37390+37400+37410+37420+37430+38+38010+38020+38021+38022+38030+38040+38050+38060+38070+38080+38090+38100+38110+38111+38112+38113+38114+38115+38310+38320+38330+38340+38350+38360+38370+38380+38390+38400+39+39010+39020+39310+39320+90+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=101&tblId=DT_1B8000I",
    #  "연간", 2019, 2021],
    # # 시군구 경제활동인구 총괄
    # ["https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T1+T2+T3+T4+T9+T5+T6+T7+T11+T8+T10+&objL1=1101+1102+1103+1104+1105+1106+1107+1108+1109+1110+1111+1112+1113+1114+1115+1116+1117+1118+1119+1120+1121+1122+1123+1124+1125+2101+2102+2103+2104+2105+2106+2107+2108+2109+2110+2111+2112+2113+2114+2115+2131+2201+2202+2203+2204+2205+2206+2207+2231+2301+2302+2304+2305+2306+2307+2308+2309+2331+2332+2401+2402+2403+2404+2405+2501+2502+2503+2504+2505+2601+2602+2603+2604+2631+3099+3101+3102+3103+3104+3105+3106+3107+3108+3109+3110+3111+3112+3113+3114+3115+3116+3117+3118+3119+3120+3121+3122+3123+3124+3125+3126+3127+3128+3132+3135+3137+3138+3199+3201+3202+3203+3204+3205+3206+3207+3231+3232+3233+3234+3235+3236+3237+3238+3239+3240+3241+3299+3301+3302+3303+3331+3332+3333+3334+3335+3336+3337+3338+3339+3399+3401+3402+3403+3404+3405+3406+3407+3408+3431+3432+3433+3434+3435+3436+3437+3438+3499+3501+3502+3503+3504+3505+3506+3531+3532+3533+3534+3535+3536+3537+3538+3599+3601+3602+3603+3604+3606+3631+3632+3633+3635+3636+3637+3638+3639+3640+3641+3642+3643+3644+3645+3646+3647+3648+3699+3701+3702+3703+3704+3705+3706+3707+3708+3709+3710+3731+3732+3733+3734+3735+3736+3737+3738+3739+3740+3741+3742+3743+3799+3803+3805+3806+3807+3808+3809+3810+3811+3831+3832+3833+3834+3835+3836+3837+3838+3839+3840+3899+3901+3902+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=H&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=101&tblId=DT_1ES3A01S",
    #  "반기", 2019, 2021],
    # # 시군구/연령별 취업자 및 고용률 (전체)
    # ["https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T00+T11+T12+T21+&objL1=1101+1102+1103+1104+1105+1106+1107+1108+1109+1110+1111+1112+1113+1114+1115+1116+1117+1118+1119+1120+1121+1122+1123+1124+1125+2101+2102+2103+2104+2105+2106+2107+2108+2109+2110+2111+2112+2113+2114+2115+2131+2201+2202+2203+2204+2205+2206+2207+2231+2301+2302+2304+2305+2306+2307+2308+2309+2331+2332+2401+2402+2403+2404+2405+2501+2502+2503+2504+2505+2601+2602+2603+2604+2631+3099+3101+3102+3103+3104+3105+3106+3107+3108+3109+3110+3111+3112+3113+3114+3115+3116+3117+3118+3119+3120+3121+3122+3123+3124+3125+3126+3127+3128+3132+3135+3137+3138+3199+3201+3202+3203+3204+3205+3206+3207+3231+3232+3233+3234+3235+3236+3237+3238+3239+3240+3241+3299+3301+3302+3303+3331+3332+3333+3334+3335+3336+3337+3338+3339+3399+3401+3402+3403+3404+3405+3406+3407+3408+3431+3432+3433+3434+3435+3436+3437+3438+3499+3501+3502+3503+3504+3505+3506+3531+3532+3533+3534+3535+3536+3537+3538+3599+3601+3602+3603+3604+3606+3631+3632+3633+3635+3636+3637+3638+3639+3640+3641+3642+3643+3644+3645+3646+3647+3648+3699+3701+3702+3703+3704+3705+3706+3707+3708+3709+3710+3731+3732+3733+3734+3735+3736+3737+3738+3739+3740+3741+3742+3743+3799+3803+3805+3806+3807+3808+3809+3810+3811+3831+3832+3833+3834+3835+3836+3837+3838+3839+3840+3899+3901+3902+&objL2=000+010+020+030+040+050+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=H&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=101&tblId=DT_1ES3A03_A01S",
    #  "반기", 2019, 2021],
    # # 시군구별 성별 일반건강검진 판정 현황
    # ["https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=001+&objL1=ALL&objL2=ALL&objL3=ALL&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=350&tblId=DT_35007_N098",
    #  "연간", 2019, 2020],
    # # 시군구/월별 사망자수
    # ["https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T1+&objL1=00+11+11010+11020+11030+11040+11050+11060+11070+11080+11090+11100+11110+11120+11130+11140+11150+11160+11170+11180+11190+11200+11210+11220+11230+11240+11250+21+21010+21020+21030+21040+21050+21060+21070+21080+21090+21100+21110+21120+21130+21140+21150+21310+22+22010+22020+22030+22040+22050+22060+22070+22310+23+23010+23020+23030+23040+23050+23060+23070+23080+23090+23310+23320+24+24010+24020+24030+24040+24050+25+25010+25020+25030+25040+25050+26+26010+26020+26030+26040+26310+29+29010+31+31010+31011+31012+31013+31014+31020+31021+31022+31023+31030+31040+31041+31042+31050+31051+31052+31053+31060+31070+31080+31090+31091+31092+31100+31101+31103+31104+31110+31120+31130+31140+31150+31160+31170+31180+31190+31191+31192+31193+31200+31210+31220+31230+31240+31250+31260+31270+31280+31320+31350+31370+31380+31102+31310+31330+31340+31360+31390+31400+32+32010+32020+32030+32040+32050+32060+32070+32310+32320+32330+32340+32350+32360+32370+32380+32390+32400+32410+33+33010+33011+33012+33020+33030+33040+33041+33042+33043+33044+33310+33320+33330+33340+33350+33360+33370+33380+33390+34+34010+34011+34012+34020+34030+34040+34050+34060+34070+34080+34310+34320+34330+34340+34350+34360+34370+34380+34390+35+35010+35011+35012+35020+35030+35040+35050+35060+35310+35320+35330+35340+35350+35360+35370+35380+36+36010+36020+36030+36040+36050+36060+36310+36320+36330+36340+36350+36360+36370+36380+36390+36400+36410+36420+36430+36440+36450+36460+36470+36480+37+37010+37011+37012+37020+37030+37040+37050+37060+37070+37080+37090+37100+37310+37320+37330+37340+37350+37360+37370+37380+37390+37400+37410+37420+37430+38+38010+38020+38021+38022+38030+38040+38050+38060+38070+38080+38090+38100+38110+38111+38112+38113+38114+38115+38310+38320+38330+38340+38350+38360+38370+38380+38390+38400+39+39010+39020+39310+39320+&objL2=0+1+2+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=101&tblId=DT_1B82A01",
    #  "연간", 2019, 2020],
    # # 시군구별 전문과목별 전문의 인력현황
    # ["https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=1635413103140071S1+&objL1=ALL&objL2=ALL&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=354&tblId=DT_HIRA4S",
    #  "분기", 2019, 2021],
    # # 시도/시군구별 평균초혼 연령
    # ["https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T10+T20+&objL1=00+11+11010+11020+11030+11040+11050+11060+11070+11080+11090+11100+11110+11120+11130+11140+11150+11160+11170+11180+11190+11200+11210+11220+11230+11240+11250+21+21010+21020+21030+21040+21050+21060+21070+21080+21090+21100+21110+21120+21130+21140+21150+21310+22+22010+22020+22030+22040+22050+22060+22070+22310+23+23010+23020+23030+23040+23050+23060+23070+23080+23090+23310+23320+24+24010+24020+24030+24040+24050+25+25010+25020+25030+25040+25050+26+26010+26020+26030+26040+26310+29+29010+31+31010+31011+31012+31013+31014+31020+31021+31022+31023+31030+31040+31041+31042+31050+31051+31052+31053+31060+31070+31080+31090+31091+31092+31100+31101+31102+31103+31104+31110+31120+31130+31140+31150+31160+31170+31180+31190+31191+31192+31193+31200+31210+31220+31230+31240+31250+31260+31270+31280+31310+31320+31330+31340+31350+31360+31370+31380+32+32010+32020+32030+32040+32050+32060+32070+32310+32320+32330+32340+32350+32360+32370+32380+32390+32400+32410+33+33010+33011+33012+33020+33030+33040+33041+33042+33043+33044+33310+33320+33330+33340+33350+33360+33370+33380+33390+34+34010+34011+34012+34020+34030+34040+34050+34060+34070+34080+34310+34320+34330+34340+34350+34360+34370+34380+34390+35+35010+35011+35012+35020+35030+35040+35050+35060+35310+35320+35330+35340+35350+35360+35370+35380+36+36010+36020+36030+36040+36060+36310+36320+36330+36350+36360+36370+36380+36390+36400+36410+36420+36430+36440+36450+36460+36470+36480+37+37010+37011+37012+37020+37030+37040+37050+37060+37070+37080+37090+37100+37310+37320+37330+37340+37350+37360+37370+37380+37390+37400+37410+37420+37430+38+38010+38020+38021+38022+38030+38040+38050+38060+38070+38080+38090+38100+38110+38111+38112+38113+38114+38115+38310+38320+38330+38340+38350+38360+38370+38380+38390+38400+39+39010+39020+39310+39320+90+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=101&tblId=DT_1B83A09",
    #  "연간", 2019, 2021],
    # # 시도/시군구/월별 이혼
    # ["https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T4+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=101&tblId=DT_1B85033",
    #  "연간", 2019, 2021],
    # # 운전면허소지자현황(연령대별)
    # ["https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=16132AAA0+&objL1=ALL&objL2=ALL&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=132&tblId=DT_13201_A002",
    #  "연간", 2019, 2021],
]

In [87]:
# 수집 실행
for url, cycle, start, end in to_collects:
    print(cycle, start, end)
    # 수집
    filename, df = collect_kosis_api_data_with_date_range(url, cycle, start, end, pass_error=False)
    print(filename)
    filename = filename.replace("/", "&")
    # 저장
    to_save = f"./DATA_KOSIS/kosis_{filename}.csv"
    df.to_csv(to_save)
    print("-" * 50)

In [91]:
# 별도수집

# 연령별/성별/혼인상태별 인구(15세이상,내국인)-시군구 (5년간 데이터, 2020년만 수집)
# 연령별로 하나씩 수집한 뒤에 하나로 합침

url_base = "https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T10+T11+T12+T13+T14+T20+T21+T22+T23+T24+T30+T31+T32+T33+T34+&objL1=ALL&objL2={agecode}+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe=2020&endPrdDe=2020&loadGubun=2&orgId=101&tblId=DT_1PM2002"
age_codes = "000+020+025+030+035+040+045+050+055+060+065+070+075+080+085+086".split("+")

dfs = []
for age in age_codes:
    url = url_base.format(agecode=age)
    
    filename, df = collect_kosis_api_data_with_date_range(url, "연간", 2020, 2020, pass_error=False)
    # 저장
    filename = filename.replace("/", "&")
    to_save = f"./DATA_KOSIS/kosis_{filename}.csv"
    dfs.append(df)
    print("-" * 50)

df = pd.concat(dfs)
df.reset_index(drop=True, inplace=True)
df.to_csv(to_save)

100%|██████████| 1/1 [00:03<00:00,  3.87s/it]


--------------------------------------------------


100%|██████████| 1/1 [00:04<00:00,  4.06s/it]


--------------------------------------------------


100%|██████████| 1/1 [00:04<00:00,  4.23s/it]


...

--------------------------------------------------


100%|██████████| 1/1 [00:04<00:00,  4.38s/it]


--------------------------------------------------


100%|██████████| 1/1 [00:04<00:00,  4.47s/it]


--------------------------------------------------


In [92]:
# 별도 수집
# 자동차등록대수현황 시도별
    # ["https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=13103873443T1+13103873443T2+13103873443T3+13103873443T4+&objL1=ALL&objL2=ALL&objL3=ALL&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=116&tblId=DT_MLTM_5498",
    #  "월간", 2019, 2021],
    
url_base = "https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId={itm_id}+&objL1=ALL&objL2=ALL&objL3=ALL&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe=202206&endPrdDe=202206&loadGubun=2&orgId=116&tblId=DT_MLTM_5498"
itm_ids = "13103873443T1+13103873443T2+13103873443T3+13103873443T4".split("+")

dfs = []
for itm_id in itm_ids:
    url = url_base.format(itm_id=itm_id)
    
    filename, df = collect_kosis_api_data_with_date_range(url, "월간", 2019, 2021, pass_error=False)
    # 저장
    filename = filename.replace("/", "&")
    to_save = f"./DATA_KOSIS/kosis_{filename}.csv"
    dfs.append(df)
    print("-" * 50)

df = pd.concat(dfs)
df.reset_index(drop=True, inplace=True)
df.to_csv(to_save)

100%|██████████| 36/36 [01:03<00:00,  1.78s/it]


--------------------------------------------------


100%|██████████| 36/36 [01:03<00:00,  1.75s/it]


--------------------------------------------------


100%|██████████| 36/36 [01:02<00:00,  1.75s/it]


--------------------------------------------------


100%|██████████| 36/36 [01:03<00:00,  1.76s/it]


--------------------------------------------------
